<a href="https://colab.research.google.com/github/GaoIzoo/AI-DEPLOYMENT/blob/main/Pr%C3%A9diction_du_cancer_du_sein_par_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Prédiction du cancer du  sein par machine learning

À propos de l'ensemble de données

Les caractéristiques sont calculées à partir d'une image numérisée d'une aspiration à l'aiguille fine (FNA) d'une masse mammaire. Elles décrivent les caractéristiques des noyaux cellulaires présents dans l'image.
n l'espace tridimensionnel est celui décrit dans : [K. P. Bennett et O. L. Mangasarian : "Robust Linear Programming Discrimination of Two Linearly Inseparable Sets", Optimization Methods and Software 1, 1992, 23-34].

Cette base de données est également disponible sur le site:
ftp ftp.cs.wisc.edu cd math-prog/cpo-dataset/machine-learn/WDBC/

Également disponible sur le dépôt d'apprentissage automatique de l'UCI : https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

Informations sur les variables :

1) Numéro d'identification
2) Diagnostic (M = malin, B = bénin)
3-32)

Dix caractéristiques à valeur réelle sont calculées pour chaque noyau cellulaire :

a) le rayon (moyenne des distances entre le centre et les points du périmètre)
b) texture (écart-type des valeurs de l'échelle de gris)
c) périmètre
d) surface
e) régularité (variation locale des longueurs de rayon)
f) compacité (périmètre^2 / surface - 1,0)
g) concavité (sévérité des parties concaves du contour)
h) points concaves (nombre de portions concaves du contour)
i) la symétrie
j) dimension fractale ("approximation du trait de côte" - 1)

La moyenne, l'erreur standard ou la plus grande (moyenne des trois plus grandes valeurs) de ces caractéristiques ont été calculées.
plus grandes valeurs) de ces caractéristiques ont été calculées pour chaque image,
ce qui donne 30 caractéristiques. Par exemple, le champ 3 est le Rayon moyen, le champ 13 est le rayon SE, le champ 23 est le rayon le plus mauvais.

Toutes les valeurs des caractéristiques sont recodées avec quatre chiffres significatifs.

Valeurs d'attributs manquantes : aucune

Distribution des classes : 357 bénignes, 212 malignes

Traduit avec www.DeepL.com/Translator (version gratuite)

# 2. Importation des packages

In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns 
import missingno as msno

import warnings
warnings.filterwarnings("ignore") 

#Pour la construction des différents modèles de ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xg

# 3. Exploration de la base de donnée

**3.1) Importation de la base**

In [2]:
# importation de la base data.csv
import pandas as pd
df = pd.read_csv ('/content/data.csv',  sep=',')
df.head()

FileNotFoundError: ignored

In [ ]:
print("La base de données sur le cancer comprend ", df.shape)

**Suppression de la variable unnamed 32 qui ne comporte que des variables manquantes**

In [ ]:
# suppression de la colonne "id" et "Unnamed: 32"
df = df.drop(df.columns[[0,32]], axis=1)
df.head(30)

**3.2) Analysons la variable à expliquer (Diagnostic)**

In [ ]:
sns.countplot(df['diagnosis'],label="Count")

In [ ]:
# nombre total de cancéreuse et non cancéreuse
df['diagnosis'].value_counts()

On a un nombre plus élévé de cancer de sein benigne que maligne, soit environ 62,7 % et 37,3% de maligne

In [ ]:
# Valeurs manquantes

df.isna().sum()
msno.bar(df)

Nous n'avons pas de valeurs manquantes dans notre base de données

**3.3 Etude la correlation entre les différentes variables**

Transformation de la variable cible en variable quantitative

In [ ]:
# Graphique de correlation

plt.figure(figsize = (20, 12))

corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

sns.heatmap(corr, mask = mask, linewidths = 1, annot = True, fmt = ".2f")
plt.show()

Nous pouvons voir qu'il y a beaucoup de colonnes qui sont très fortement corrélées, ce qui provoque la multicollinéarité. Nous devons donc éliminer les caractéristiques fortement corrélées.

In [ ]:
# removing highly correlated features

corr_matrix = df.corr().abs() 

mask = np.triu(np.ones_like(corr_matrix, dtype = bool))
tri_df = corr_matrix.mask(mask)

to_drop = [x for x in tri_df.columns if any(tri_df[x] > 0.6)]

df = df.drop(to_drop, axis = 1)

print(f"La base de données a été reduit à {df.shape[1]} columns.")

In [ ]:
df.head()

In [ ]:
# remplaçons M ( cancer maligne ) par 1 et B ( cancer benigne ) par 0 sur la colonne "diagnosis"
df.diagnosis = df.diagnosis.replace({"M":1,"B":0})
df.head()

In [ ]:
df.to_csv()

Nous ne garderons que 

# 4. Implémentation des différents modèles de ML

**4.1 Subdivision de la base de données en données d'apprentissage et test**

Divsion en 70 % de données d'apprentissage et 20 % de test

In [ ]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']

In [ ]:
# splitting data into training and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

**4.2) Normalisation de nos données**

Nous remarquons une très grande variabilté en nos variables. Nous allons proceder à la normalisation de nos données de test.

In [ ]:
# scaling data

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

**4.3) Implémentation**

4.3.1) Regression Logistique

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
predictions1 = logreg.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))
print('\n')
print(classification_report(y_test, predictions1))

In [ ]:
from sklearn.metrics import accuracy_score

logreg_acc = accuracy_score(y_test, predictions1)
print("Accuracy of the Logistic Regression Model is: ", logreg_acc)

4.3.2) Support Vector Classifier (SVC)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc = SVC()
parameters = {
    'gamma' : [0.0001, 0.001, 0.01, 0.1],
    'C' : [0.01, 0.05, 0.5, 0.1, 1, 10, 15, 20]
}

grid_search = GridSearchCV(svc, parameters)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
svc = SVC(C = 15, gamma = 0.01)
svc.fit(X_train, y_train)

In [ ]:
y_pred = svc.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, svc.predict(X_train)))

svc_acc = accuracy_score(y_test, svc.predict(X_test))
print(svc_acc)

In [ ]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

In [ ]:
# classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

4.3.3)  ANN (neural network )

In [ ]:
np.random.seed(42)
import tensorflow as tf

# Initialising the ANN

ann = tf.keras.models.Sequential()

# Adding the input layer and the first hidden layer

ann.add(tf.keras.layers.Dense(units = 7, activation = 'relu'))

# Adding the second hidden layer

ann.add(tf.keras.layers.Dense(units = 7, activation = 'relu'))
# Adding the third hidden layer

ann.add(tf.keras.layers.Dense(units = 7, activation = 'relu'))

# Adding the fourth hidden layer

ann.add(tf.keras.layers.Dense(units = 7, activation = 'relu'))

# Adding the output layer

ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
# Compiling the ANN

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy' , metrics = ['accuracy'] )

# Training the ANN on the training set

ann.fit(X_train, y_train, batch_size = 16, epochs = 100)

In [ ]:
# Predicting the test set results

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.9)
np.set_printoptions()

In [ ]:
# Making the confusion matrix, calculating accuracy_score 

from sklearn.metrics import confusion_matrix, accuracy_score

# confusion matrix
cm = confusion_matrix(y_test,y_pred)
print("Confusion Matrix")
print(cm)

In [ ]:
# accuracy
ac_ann = accuracy_score(y_test,y_pred)
print("Accuracy")
print(ac_ann)

4.3.4) Les forêts aléatoires (Random Forests)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rand_clf = RandomForestClassifier(criterion = 'entropy', max_depth = 11, max_features = 'auto', min_samples_leaf = 2, min_samples_split = 3, n_estimators = 130)
rand_clf.fit(X_train, y_train)

In [ ]:
y_pred = rand_clf.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, rand_clf.predict(X_train)))

ran_clf_acc = accuracy_score(y_test, y_pred)
print(ran_clf_acc)

In [ ]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

4.3.5) Support vector Machine 

In [ ]:
from sklearn.svm import SVC

svc_model = SVC(kernel="rbf")
svc_model.fit(X_train, y_train)
predictions3 = svc_model.predict(X_test)

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predictions3))
print("\n")
print(classification_report(y_test, predictions3))

In [ ]:
svm_acc = accuracy_score(y_test, predictions3)
print("Accuracy of SVM model is: ", svm_acc)

# 5. Benchmark des différents modèles

5.1 Visualisation de différents resultats

In [ ]:
mylist=[]
mylist2=[]
mylist.append(svc_acc)
mylist2.append("SVC")
mylist.append(svc_acc)
mylist2.append("SVC")
mylist.append(logreg_acc)
mylist2.append("Logistic Regression")
mylist.append(ran_clf_acc)
mylist2.append("Random Forests")
mylist.append(svm_acc)
mylist2.append("Support Vector Machines")
mylist.append(ac_ann)
mylist2.append("ANN")
plt.rcParams['figure.figsize']=22,10
sns.set_style("darkgrid")
ax = sns.barplot(x=mylist2, y=mylist, palette = "coolwarm", saturation =1.5)
plt.xlabel("Classification Models", fontsize = 20 )
plt.ylabel("Accuracy", fontsize = 20)
plt.title("Accuracy of different Classification Models", fontsize = 20)
plt.xticks(fontsize = 11, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 13)
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.2%}', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
plt.show()

Le meilleur modèle  en terme d'accuracy est le SVC

**Testons les performances du SVC**

# Sauvegarde et importation de notre modèle

In [ ]:
import pickle

**Enregistrer mon modèle**

In [ ]:
df.head()

In [ ]:
filename = 'trained_model.sav'
pickle.dump(svc, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [ ]:
input_data = (0.7452,1003,0.1,0.002897,23.87,0.22,0.2908,0.07277)
input_data_as_numpy_array = np.asarray(input_data)
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)
prediction = loaded_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('La personne a un cancer benigne')
else:
  print('La personne a un cancer maligne')


[1]
La personne a un cancer maligne
